In [1]:
# cell [1]
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('drive')
import pandas as pd

Mounted at drive


VEP input needs to be in VCF format. Otherwise, VEP may output warnings or have missing variants. You'll need to run VEP again with the missing variant in VCF format, which includes its location.

In [3]:
# cell [2]
# Annotations must be VEP output. Replace with appropriate file location for filepath and file name for filename.
filepath = "/content/drive/MyDrive/data/arboleda/"
filename = "ESM1b_test_vep.txt"
file_name_i = filepath + filename

# Skips rows until reaching header of VEP output.
skip = 0
with open(file_name_i,'r') as data_file:
  data = data_file.readlines()
for line in data:
  if "#Uploaded_variation" not in line:
    skip += 1
  if "#Uploaded_variation" in line:
    break
annotations = pd.read_csv(file_name_i, sep="\t", skiprows=skip)
annotations.head()

# Reformats UniProtKB IDs for variant isoforms that end in "-1".
for i in range (0, len(annotations)):
  if (annotations.loc[i, "UNIPROT_ISOFORM"] != "-" and annotations.loc[i, "UNIPROT_ISOFORM"][-2:] == "-1"):
    annotations.loc[i, "UNIPROT_ISOFORM"] = annotations.loc[i, "UNIPROT_ISOFORM"][:-2]
annotations = annotations.reset_index(drop=True)

# Creates a new column for protein change to be used in ESM1b input.
annotations["Protein_change"] = ""
for i in range(0, len(annotations)):
  if (("/" in annotations["Amino_acids"][i]) and (annotations["Protein_position"][i] != "-")):
    protein_change = annotations["Amino_acids"][i].split("/")[0] + annotations["Protein_position"][i] + annotations["Amino_acids"][i].split("/")[1]
    annotations.loc[i, "Protein_change"] = protein_change

# Creates a list of ENSP IDs for variants missing UniProtKB IDs for BioMart input.
missing_uniprot = annotations.drop_duplicates(subset="ENSP")
missing_uniprot = missing_uniprot[missing_uniprot["UNIPROT_ISOFORM"] == "-"]
missing_uniprot = missing_uniprot[missing_uniprot["ENSP"] != "-"] ## FIX ME: No entry for variants that don't have ENSP IDs ##
missing_uniprot = missing_uniprot.reset_index(drop=True)

# Manually upload file with missing UniProtKB IDs into BioMart with proper settings.
output = missing_uniprot["ENSP"]
output.to_csv(filepath + "biomart.csv", header=False, index=False)
# http://useast.ensembl.org/biomart/martview/

Ensembl Genes 112 > Human genes (GRCh38.p14)

Filters > Gene > Input external references ID list [Max 500 advised] > Protein stable ID(s) [e.g. ENSP00000000233] > Choose File > biomart.csv

Attributes > Gene > Ensembl > Protein stable ID

Uncheck Gene stable ID, Gene stable ID version, Transcript stable ID, Transcript stable ID version

Attributes > External > UniProtKB Gene Name ID, UniProtKB isoform ID

Results > Unique results only > Go

In [4]:
# cell [3]
# Put retrieved UniProtKB IDs from BioMart back into VEP annotation.
found_names = pd.read_csv(filepath + "mart_export.txt", sep="\t")

# Prioritizes UniProtKB Isoform IDs, otherwise uses UniProtKB Gene IDs.
for i in range (0, len(found_names)):
  if pd.isna(found_names["UniProtKB isoform ID"][i]):
    found_names.loc[i, "UniProtKB isoform ID"] = found_names.loc[i, "UniProtKB Gene Name ID"]

found_names = found_names.drop("UniProtKB Gene Name ID", axis=1)
found_names.dropna(subset="UniProtKB isoform ID").reset_index(drop=True)
found_names.sort_values("Protein stable ID").reset_index(drop=True)

# Creates a list of UniProtKB IDs outputted for one ENSP ID and matches it back to VEP annotations.
uniprot = ""
for i in range (0, len(found_names)-1):
  old_name = found_names["Protein stable ID"][i]
  new_name = found_names["Protein stable ID"][i+1]
  if new_name != old_name and type(found_names["UniProtKB isoform ID"][i]) != float:
    uniprot += found_names["UniProtKB isoform ID"][i]
    index = annotations.index[annotations["ENSP"]==old_name].tolist()
    for j in range(0, len(index)):
      if (annotations.loc[index[j], "UNIPROT_ISOFORM"] == "-"):
        annotations.loc[index[j], "UNIPROT_ISOFORM"] = uniprot
    uniprot = ""
  else:
    if type(found_names["UniProtKB isoform ID"][i]) != float:
      uniprot += found_names["UniProtKB isoform ID"][i] + ", "
uniprot += found_names["UniProtKB isoform ID"][len(found_names)-1]
index = annotations.index[annotations["ENSP"]==old_name].tolist()
for j in range(0, len(index)):
  if (annotations.loc[index[j], "UNIPROT_ISOFORM"] == "-"):
    annotations.loc[index[j], "UNIPROT_ISOFORM"] = uniprot

# Takes each UniProtKB ID for each entry and associated protein change to get ESM1b scores.
df = pd.DataFrame(columns=["uniprot_id", "protein_change"])
for i in range(0, len(annotations)):
  if ("/" in annotations["Amino_acids"][i]) and (annotations["Protein_position"][i] != "-") and (annotations["UNIPROT_ISOFORM"][i] != "-"):
    IDs = annotations["UNIPROT_ISOFORM"][i].split(", ")
    protein = annotations["Amino_acids"][i].split("/")[0] + annotations["Protein_position"][i] + annotations["Amino_acids"][i].split("/")[1]
  for j in range(0, len(IDs)):
    new_row = [IDs[j], protein]
    df.loc[len(df)] = new_row
df = df.drop_duplicates().reset_index(drop=True)

# Output to run through pipeline on Hoffman2 for ESM1b scores.
df.to_csv(filepath + "run_ESM1b.csv", index=False)
# Upload run_ESM1b to Hoffman2 and ./get_ESM1b.

In [5]:
# cell [4]
# Puts most severe ESM1b scores into new column in VEP annotations.
ESM1b_scores = pd.read_csv(filepath + "ESM1b_scores.csv")
annotations["ESM1b"] = '-'
for i in range(0, len(ESM1b_scores)):
  uniprot_id = ESM1b_scores["uniprot_id"][i]
  protein_change = ESM1b_scores["protein_change"][i]
  score = ESM1b_scores["score"][i]
  index = annotations.index[annotations["Protein_change"]==protein_change].tolist()
  for j in range(0, len(index)):
    if uniprot_id in annotations.loc[index[j], "UNIPROT_ISOFORM"]:
      if (annotations["ESM1b"][index[j]] != '-'):
        if score < annotations.loc[index[j], "ESM1b"]:
          annotations.loc[index[j], "ESM1b"] = score
      else:
        if (pd.isna(score) != True):
          annotations.loc[index[j], "ESM1b"] = score

# Replace entries with empty ESM1b scores with '-'.
for i in range(0, len(annotations)):
  if pd.isna(annotations["ESM1b"][i]):
    annotations.loc[i, "ESM1b"] = '-'

# Delete extra Protein_change column.
annotations = annotations.drop('Protein_change', axis=1)

# Final VEP annotations with column for ESM1b scores.
file_name_o = filepath + filename.split(".")[0] + "_v2.csv"
annotations.to_csv(file_name_o, index=False)